### 1.  Creating a new MariaDB database "PresidentDB" with user "President_User" and granting it access 

create database PresidentDB; 
grant all on PresidentDB.* to 'President_User' identified by 'President_Python'; 
flush privileges; 
quit;

### Creating table structure to store web scraped data

mysql -u President_User -p PresidentDB;
Enter password: 

create table PresidentsInfo(
     Name varchar(200) ,
     TOO_StrtDate varchar(100),
     TOO_EndDate varchar(100) ,
     Party varchar(50) ,
     Tenure varchar(20),
     EC_CompParty varchar(80),
     VicePresident varchar(50),
     Dale varchar(30),
     PresidentName varchar(50),
     Age varchar(30),
     Constituency1 varchar(200),
     Constituency2 varchar(200) );

###  2 . Python code to scrape the wikipedia table using Pandas: 

In [ ]:
URL = "https://en.wikipedia.org/wiki/Taoiseach"
from gazpacho import Soup,get
import pandas as pd
html = get(URL)

soup = Soup(html)

In [ ]:
df_Presidents = pd.read_html(str(soup.find('table')))[3]
df_Presidents= df_Presidents.loc[1:,2:]
df_Presidents

## Data cleaning on the above created data frame . 

### Creating columns by naming them into the dataframe:

In [ ]:
df_Presidents.columns=['Name','TermofOfficeStartDate','TermoffficeEndDate','Party','Tenure','Exec.CouCompParty','vp','VicePresident','Dail']

In [ ]:
df_Presidents

### Removing rows 10 and 11 as they do not contain meaningful data 

In [ ]:
df_Presidents = df_Presidents.loc[2:,]
df_Presidents = df_Presidents.drop([10,11],axis = 0)
df_Presidents

###  Splitting the "Name" variable and extracting only name, age and constituency from it.

In [ ]:
df_Presidents["President_Name"] = df_Presidents["Name"].str.split("(",expand = True)[0]
df_Presidents.head(10)

In [ ]:
df_Presidents["Age"] = df_Presidents["Name"].str.split("(",expand = True)[1].str.split(")",expand= True)[0]





### Splitting constituency variable into two variables constituency1 and constituency2 : 

In [ ]:
df_Presidents["Constituency"] = df_Presidents["Name"].str.split("(",expand = True)[1].str.split(")",expand= True)[1]

df_Presidents["Constituency1"] = df_Presidents["Constituency"].str.split("TD for",expand = True)[1]

df_Presidents["Constituency2"] = df_Presidents["Constituency"].str.split("TD for",expand = True)[2]

### Removing columns "vp" and "Constituency"

In [ ]:
df_Presidents = df_Presidents.drop('vp', axis = 1)

df_Presidents = df_Presidents.drop('Constituency', axis = 1) 



df_Presidents.head(5)

### Re - arranging the index of the dataframe 

In [ ]:
import numpy as np
df_Presidents.index = np.arange(1,len(df_Presidents) + 1)
df_Presidents

### Cleaning TermofOfficeStartDate and TermofOfficeEndDate columns as it has unwanted text 

In [ ]:
df_Presidents['TermofOfficeStartDate'] = df_Presidents['TermofOfficeStartDate'].str.split('[',expand = True)[0]

df_Presidents['TermoffficeEndDate'] = df_Presidents['TermoffficeEndDate'].str.split('[',expand = True)[0]


In [ ]:
df_Presidents

In [ ]:
from datetime import datetime

df_Presidents = df_Presidents.replace(df_Presidents.loc[44]['TermoffficeEndDate'],datetime.now())



In [ ]:
df_Presidents 

### Converting the columns TermofOfficeStartDate and TermoffficeEndDate to "datetime" type from datetime module.

In [ ]:





df_Presidents['TermofOfficeStartDate'] = pd.to_datetime(df_Presidents['TermofOfficeStartDate'])





df_Presidents['TermoffficeEndDate'] = pd.to_datetime(df_Presidents['TermoffficeEndDate'])

df_Presidents['TermoffficeEndDate'] = df_Presidents['TermoffficeEndDate'].astype(str)

df_Presidents['TermoffficeEndDate'] = df_Presidents['TermoffficeEndDate'].str.split(' ', expand = True)[0]

df_Presidents['TermoffficeEndDate'] 

df_Presidents['TermoffficeEndDate'] = pd.to_datetime(df_Presidents['TermoffficeEndDate'])

df_Presidents


### Configuring the connection settings to the database:

In [ ]:
config = {
    'user' : 'President_User',
    'password' : 'President_Python',
    'host' : '127.0.0.1',
    'database' : 'PresidentDB'
}

In [ ]:
import DBcm

### Using DBcm module inserting dataframe into table "PresidentInfo"

In [ ]:
with DBcm.UseDatabase(config) as Cursor:
    SQL = """INSERT INTO PresidentsInfo(Name,TOO_StrtDate,TOO_EndDate,Party,Tenure,EC_CompParty,VicePresident,Dale,PresidentName,Age,Constituency1,Constituency2) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    i=0
    while i < len(df_Presidents):
        data = [row for row in df_Presidents.iloc[i]]
        data1 = [tuple(data)]
        Cursor.executemany(SQL,data1)
        i+=1
        

### 3. a ] List of constituencies in Ireland that have produced a President. For this we need to clean data in table for columns constituency1 and constituency 2 which have extra text like year specified in it.


update PresidentsInfo set Constituency1 = ' Carlow–Kilkenny' where Constituency1 like '%1927';

update PresidentsInfo set Constituency1 = ' Cork Borough' where Constituency1 like '%1969';

update PresidentsInfo set Constituency2 = ' Cork Borough' where Constituency2 like '%1927'; 

update PresidentsInfo set Constituency2 = ' Cork City North-West' where Constituency2 like '%1969'; 


Query :

select distinct Constituency1, Constituency2 from PresidentsInfo;

+------------------------------+-----------------------+
| Constituency1                | Constituency2         |
+------------------------------+-----------------------+
|  Carlow–Kilkenny             |  Cork Borough         |
|  Clare                       | NULL                  |
|  Dublin South-East           | NULL                  |
|  Dublin South-Central        | NULL                  |
|  Cork Borough                |  Cork City North-West |
|  Dún Laoghaire and Rathdown  | NULL                  |
|  Cork City                   | NULL                  |
|  Dublin Artane               | NULL                  |
|  Dublin North-Central        | NULL                  |
|  Longford–Roscommon          | NULL                  |
|  Meath                       | NULL                  |
|  Dublin Central              | NULL                  |
|  Laois–Offaly                | NULL                  |
|  Mayo                        | NULL                  |
|  Dublin West                 | NULL                  |
+------------------------------+-----------------------+
15 rows in set (0.000 sec)


### 3 b ]  Political party that has produced most Presidents in Ireland

Query :

select Party, count(distinct PresidentName) as Presidents_Count from PresidentsInfo group by Party order by count(distinct PresidentName) desc;


+------------------------+------------------+
| Party                  | Presidents_Count |
+------------------------+------------------+
| Fianna Fáil            |                7 |
| Fine Gael              |                6 |
| Cumann na nGaedheal    |                1 |
| Sinn Féin(Pro-Treaty)  |                1 |
+------------------------+------------------+
4 rows in set (0.001 sec)


select Party, count(distinct PresidentName) as Presidents_Count from PresidentsInfo group by Party order by count(distinct PresidentName) desc limit 1;

+--------------+------------------+
| Party        | Presidents_Count |
+--------------+------------------+
| Fianna Fáil  |                7 |
+--------------+------------------+
1 row in set (0.001 sec)



### 3 c ] List of Politicians who previously held the position of vice president before serving as a President.

 ### Creating another dataframe which is subset of df_Presidents. Using logic of nested for loop on Presidents and vice President columns to check the condition of President who is vice president priorly. 

In [ ]:
df_Presidents_Subset = df_Presidents[["TermoffficeEndDate","VicePresident","President_Name"]]

df_Presidents_Subset 

In [ ]:
data = []
i=0
j=0
bflag = False
for i in range(0,len(df_Presidents_Subset)-1):
    
    for j in range(0,len(df_Presidents_Subset)-2):
        
        if i< len(df_Presidents_Subset) and j< len(df_Presidents_Subset):
            
            if df_Presidents_Subset.iloc[i]["VicePresident"] == df_Presidents_Subset.iloc[j+1]["President_Name"]:
                
                if data != df_Presidents_Subset.iloc[i]["VicePresident"]:
                    
                    data.append(df_Presidents_Subset.iloc[i]["VicePresident"])
                    
                    bflag = True
                    
                    if bflag == True:
                        
                        break

In [ ]:
data = set(data)
data

In [ ]:
List_Presidents_VP = list(data)

List_Presidents_VP 

###  3 d ] To find political party that held the office of President for longest time. To compute the duration we took the difference of startdate and enddate and then subdivided the years into months and days using arithmetic logic.

In [ ]:
df_Presidents_Duration = df_Presidents[["President_Name","VicePresident","Party","TermofOfficeStartDate","TermoffficeEndDate"]]
df_Presidents_Duration

### Taking difference of Enddate and StartDate and then splitting the value to get only year value(whole without decimal).

In [ ]:
duration = ((df_Presidents_Duration['TermoffficeEndDate'] - df_Presidents_Duration['TermofOfficeStartDate'])/np.timedelta64(1,'Y'))

duration_years = duration.astype(str)

df_Presidents_Duration['Years'] = duration_years.str.split('.',expand = True)[0]

df_Presidents_Duration

### Extracting month value from years value obtained above using arithmetic computation: 

In [ ]:
month = duration_years.str.split('.',expand = True)[1]

month2 = "0." +month

month3 = month2.astype(float)

month_val = month3*12

month_val

month_val = month_val.astype(str)

upd_month_val = month_val.str.split('.',expand = True)

df_Presidents_Duration['Months'] = upd_month_val[0]

df_Presidents_Duration

### Extracting days from the month value remainder. Also incrementing day by 1 if remainder of days is greater than 0.2 

In [ ]:
upd_month_val[1]

days = upd_month_val[1]

days_new = "0." + days

days_update = days_new.astype(float)

days_final = days_update *30

days_final

days_strval = days_final.astype(str)

days_splitted = days_strval.str.split('.',expand = True)

day_1 = days_splitted[0]
day_1 = day_1.astype(float)
day_2 = days_splitted[1]

day_3 = "0." +day_2

day_3 = day_3.astype(float)

for i in range(1,46):
    if day_3[i] >= 0.2:
        day_1[i] = day_1[i] + 1
    else:
        day_1[i] = day_1[i]
        
day_1

df_Presidents_Duration['Days'] = day_1

df_Presidents_Duration


### Converting Years and months columns to Float type :

In [ ]:
df_Presidents_Duration['Years'] = df_Presidents_Duration['Years'].astype(float)

df_Presidents_Duration['Months'] = df_Presidents_Duration['Months'].astype(float)

df_Presidents_Duration

### Creating  a table President_Duration and inserting above data into table "President_Duration" 

create table President_Duration(
    ->      PresidentName varchar(200) ,
    ->      VicePresident varchar(100),
    ->      Party varchar(100),
    ->      TOO_StrtDate varchar(100) ,
    ->      TOO_EndDate varchar(50) ,
    ->      Years varchar(20),
    ->      Months varchar(80),
    ->      Days varchar(50)
    ->       );
Query OK, 0 rows affected (0.172 sec)


In [ ]:
config = {
    'user' : 'President_User',
    'password' : 'President_Python',
    'host' : '127.0.0.1',
    'database' : 'PresidentDB'
}

import DBcm

with DBcm.UseDatabase(config) as Cursor:
    SQL = """INSERT INTO President_Duration(PresidentName,VicePresident,Party,TOO_StrtDate,TOO_EndDate,Years,Months,Days) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
    i=0
    while i < len(df_Presidents_Duration):
        data = [row for row in df_Presidents_Duration.iloc[i]]
        data1 = [tuple(data)]
        Cursor.executemany(SQL,data1)
        i+=1

### Creating temporary table to find distinct presidents and then executing a query to find which party has ruled for the longest duration 

Query:

create temporary table President_Temp as ( select distinct PresidentName, Party , TOO_StrtDate, TOO_EndDate,Years,Months,Days from President_Duration);

select Party, sum(Years) as Years, sum( Months ) as Months, sum(Days) as Days from President_Temp group by Party order by sum(Years) desc;

+------------------------+-------+--------+------+
| Party                  | Years | Months | Days |
+------------------------+-------+--------+------+
| Fianna Fáil            |    53 |     93 |  136 |
| Fine Gael              |    23 |     39 |  129 |
| Cumann na nGaedheal    |     9 |      3 |    3 |
| Sinn Féin(Pro-Treaty)  |     9 |      3 |    3 |
+------------------------+-------+--------+------+
4 rows in set (0.000 sec)


select Party, sum(Years) as Years, sum( Months ) as Months, sum(Days) as Days from President_Temp group by Party order by sum(Years) desc limit 1;

+--------------+-------+--------+------+
| Party        | Years | Months | Days |
+--------------+-------+--------+------+
| Fianna Fáil  |    53 |     93 |  136 |
+--------------+-------+--------+------+
1 row in set (0.000 sec)


### 3 e ]  To find which politician held the office of President for longest amount uninterrupted:

### Using logic to compare (Condition for uninterrupted rule) if the current and next president from the column list is equal and also term Office end date for current and startdate for next is equal then perform the summation of years for that current president.

In [ ]:
for i in range(0,len(df_Presidents_Duration)-1):
    if df_Presidents_Duration.iloc[i]['President_Name'] == df_Presidents_Duration.iloc[i+1]['President_Name'] and df_Presidents_Duration.iloc[i]['TermoffficeEndDate'] ==  df_Presidents_Duration.iloc[i+1]['TermofOfficeStartDate']:
        
        var_PresidentName = (df_Presidents_Duration.iloc[i]['President_Name'])
        var_years = (int(df_Presidents_Duration.iloc[i]['Years'])+ int(df_Presidents_Duration.iloc[i+1]['Years']))
        var_months = (int(df_Presidents_Duration.iloc[i]['Months'])+ int(df_Presidents_Duration.iloc[i+1]['Months']))
        var_days = (int(df_Presidents_Duration.iloc[i]['Days'])+ int(df_Presidents_Duration.iloc[i+1]['Days']))
        
        print(var_PresidentName,'-' ,var_years,"Years", " " , var_months , "Months" , " ", var_days  ," " "Days ")
        
        

###  3 f ] To find which politician held the office of president for shortest time:

### Intially creating a temporary table of distinct presidents and then extracting duration of rule for each president to get shortest duration in years, months and days.

Query:

create temporary table President_Temp as ( select distinct PresidentName, Party , TOO_StrtDate, TOO_EndDate,Years,Months,Days from President_Duration);

select PresidentName,sum(Years) as Years,sum(Months) as Months,sum(Days) as Days from President_Temp group by PresidentName order by sum(Years),sum(Months);


+-------------------+-------+--------+------+
| PresidentName     | Years | Months | Days |
+-------------------+-------+--------+------+
| Leo Varadkar      |     2 |      5 |   14 |
| John Bruton       |     2 |      6 |   11 |
| Brian Cowen       |     2 |     10 |    1 |
| Albert Reynolds   |     2 |     10 |    3 |
| Liam Cosgrave     |     4 |      3 |   22 |
| Garret FitzGerald |     4 |     10 |   34 |
| John A. Costello  |     5 |     12 |   42 |
| Charles Haughey   |     5 |     26 |   28 |
| Enda Kenny        |     6 |      3 |    6 |
| Seán Lemass       |     7 |      4 |   19 |
| Jack Lynch        |     8 |      9 |   10 |
| Bertie Ahern      |    10 |     10 |   11 |
| W. T. Cosgrave    |    18 |      6 |    6 |
| Éamon de Valera   |    19 |     24 |   64 |
+-------------------+-------+--------+------+
14 rows in set (0.001 sec)

select PresidentName,sum(Years) as Years,sum(Months) as Months,sum(Days) as Days from President_Temp group by PresidentName order by sum(Years),sum(Months) limit 1;

+---------------+-------+--------+------+
| PresidentName | Years | Months | Days |
+---------------+-------+--------+------+
| Leo Varadkar  |     2 |      5 |   14 |
+---------------+-------+--------+------+
1 row in set (0.001 sec)



### 3 g ] To find political party which has most number of dails :

Query:

select Party, count(Dale) from PresidentsInfo group by Party order by count(Dale) desc;

+------------------------+-------------+
| Party                  | count(Dale) |
+------------------------+-------------+
| Fianna Fáil            |          28 |
| Fine Gael              |          12 |
| Cumann na nGaedheal    |           4 |
| Sinn Féin(Pro-Treaty)  |           1 |
+------------------------+-------------+
4 rows in set (0.001 sec)

select Party, count(Dale) from PresidentsInfo group by Party order by count(Dale) desc limit 1;
+--------------+-------------+
| Party        | count(Dale) |
+--------------+-------------+
| Fianna Fáil  |          28 |
+--------------+-------------+
1 row in set (0.002 sec)



### 3 h ] To find out which politician has office of President for largetst number of dails:

Query:

select PresidentName, count(Dale) from PresidentsInfo group by PresidentName order by count(Dale) desc limit 1;

+------------------+-------------+
| PresidentName    | count(Dale) |
+------------------+-------------+
| Éamon de Valera  |           9 |
+------------------+-------------+
1 row in set (0.001 sec)


### 3 i ] To find age of Presidents at the time they assumed office: 

### To find age we get the difference of office start date and age columns using substr function

Query:
    
select distinct PresidentName , case when Age like 'b.%' then substr(TOO_StrtDate,1,4) - substr(Age,4,4) else substr(TOO_StrtDate,1,4) - substr(Age,1,4) end as Age_StartOfice from PresidentsInfo group by PresidentName;

+-------------------+----------------+
| PresidentName     | Age_StartOfice |
+-------------------+----------------+
| Albert Reynolds   |             60 |
| Bertie Ahern      |             46 |
| Brian Cowen       |             48 |
| Charles Haughey   |             54 |
| Éamon de Valera   |             50 |
| Enda Kenny        |             60 |
| Garret FitzGerald |             55 |
| Jack Lynch        |             49 |
| John A. Costello  |             57 |
| John Bruton       |             47 |
| Leo Varadkar      |             38 |
| Liam Cosgrave     |             53 |
| Seán Lemass       |             60 |
| W. T. Cosgrave    |             42 |
+-------------------+----------------+
14 rows in set (0.001 sec)


### 3 j ] To find age of Presidents on the last day of their service:

Query:

select distinct PresidentName , case when Age like 'b.%' then substr(max(TOO_EndDate),1,4) - substr(Age,4,4) else substr(max(TOO_EndDate),1,4) - substr(Age,1,4) end as Age_on_EndofOffice from PresidentsInfo group by PresidentName order by Age_on_EndofOffice desc;

+-------------------+--------------------+
| PresidentName     | Age_on_EndofOffice |
+-------------------+--------------------+
| Éamon de Valera   |                 77 |
| Seán Lemass       |                 67 |
| Charles Haughey   |                 67 |
| John A. Costello  |                 66 |
| Enda Kenny        |                 66 |
| Jack Lynch        |                 62 |
| Albert Reynolds   |                 62 |
| Garret FitzGerald |                 61 |
| Liam Cosgrave     |                 57 |
| Bertie Ahern      |                 57 |
| W. T. Cosgrave    |                 52 |
| Brian Cowen       |                 51 |
| John Bruton       |                 50 |
| Leo Varadkar      |                 40 |
+-------------------+--------------------+
14 rows in set (0.001 sec)


### 3 k ] Statistics showing which Irish Political party is most successful :

Query :

select count(distinct PresidentName), Party from PresidentsInfo group by Party;
+-------------------------------+------------------------+
| count(distinct PresidentName) | Party                  |
+-------------------------------+------------------------+
|                             1 | Cumann na nGaedheal    |
|                             7 | Fianna Fáil            |
|                             6 | Fine Gael              |
|                             1 | Sinn Féin(Pro-Treaty)  |
+-------------------------------+------------------------+
4 rows in set (0.000 sec)

select count(distinct PresidentName) from PresidentsInfo;
+-------------------------------+
| count(distinct PresidentName) |
+-------------------------------+
|                            14 |
+-------------------------------+
1 row in set (0.000 sec)


As per the above query results it is clear that Fianna Fail has 7 presidents elected out of 14 presidents till date, ie 50% 

Query:

create temporary table President_Temp as ( select distinct PresidentName, Party , TOO_StrtDate, TOO_EndDate,Years,Months,Days from President_Duration);

select Party, sum(Years) as Years, sum( Months ) as Months, sum(Days) as Days from President_Temp group by Party order by sum(Years) desc;

+------------------------+-------+--------+------+
| Party                  | Years | Months | Days |
+------------------------+-------+--------+------+
| Fianna Fáil            |    53 |     93 |  136 |
| Fine Gael              |    23 |     39 |  129 |
| Cumann na nGaedheal    |     9 |      3 |    3 |
| Sinn Féin(Pro-Treaty)  |     9 |      3 |    3 |
+------------------------+-------+--------+------+
4 rows in set (0.000 sec)

Also we can infer from above statistics that Fianna Fail has ruled Ireland for 53 years and more from period 1922 - 2019 ( 97 years) so out of 97 years the party has won for 53 years which is 54% of the total elections. 

So from all above statistics we can conclude that Fianna Fail is most successful political party in Ireland.